# 视频教程和网页教程不能及时更新，一切以script为准！
## 注意：不要在这里运行任何代码，可copy到你们的文件夹下面使用

# 运行以下代码块进行环境配置
### 无需修改，运行即可

In [ ]:
import importlib.util
import sys
import os 
import pandas as pd
import numpy as np
from scipy.io import mmread, mmwrite

sys.path.append('/home/ztr/abio_database_pipeline/')
from pipeline.datasets_curation import datasetBuilder
from pipeline.datasets_curation import inspector
my_builder = datasetBuilder.DatasetBuilder()
my_inspector = inspector.Inspector()

In [ ]:
my_builder.buildDataset()

In [ ]:
expressionMatrix_rawCounts = 'expressionMatrix_rawCounts.tsv'
expressionMatrix_normalized = 'expressionMatrix_normalized.tsv'
expressionMatrix_TPM = 'expressionMatrix_TPM.tsv'
cellAnnotation = 'cellAnnotation.tsv'
geneAnnotation = 'geneAnnotation.tsv'
unstructuredData = 'unstructuredData.json'
readme = 'README.json'
notAvailable = 'notAvailable'

<br><br><br><br><br><br><br><br><br><br><br>

# 1. 填写expressionMatrix_rawCounts.tsv

#### 矩阵填写注意事项！（适用于以下三种矩阵）：
矩阵的行名是cellID,列名可以是geneSymbol或者ensemblID，注意如果是ensemblID，需要去除版本号，即去掉小数点及小数点后面的数字。

### 1.1 读入标准模板查看格式

In [ ]:
df_raw = my_builder.read_template_tsv(expressionMatrix_rawCounts)
df_raw.head()

### 编写完成后应如下图所示
### 列名中，第一列为cellID, 其余列为基因名。

![title](http://118.190.148.166/biodb/dataset_curation/static/6.png)

### 1.2 从 ../downloaded_data/ 路径中读取原始数据，进行清洗

In [ ]:
df_downloaded = pd.read_csv('../downloaded_data/GSM2781556_Dropseq_120000_umi.txt.gz',sep='\t')
df_downloaded.head()

### 1.3 清洗完成后，确认当前格式与目标格式相同，存储数据

In [ ]:
my_builder.save_template_tsv(df_raw, expressionMatrix_rawCounts)

### 1.4 存储为sparse matrix格式
#### (tsv和mtx格式选择一种保存即可，即如果已经保存了tsv格式则不需要存为mtx格式，反之一样)

In [ ]:
from scipy import sparse, io

#### sparse matrix 与 numpy array 互相转换：
numpy array 转换为 coo_matrix: data = sparse.coo_matrix(data)

sparse matrix 转换为: data = data.to_array()

#### 输入参数要求：
mtx为sparse coo matrix；genes和cellID为list格式，且不能有重复；genes可为symbol或者ensemblID.

In [ ]:
my_builder.save_template_mtx(mtx=, mtx_name='expressionMatrix_rawCounts.mtx', genes=, cellID=)

<br><br><br><br><br><br><br><br><br>

# 2. 填写expressionMatrix_normalized.tsv

### 2.1 读入标准模板查看格式

In [ ]:
df_norm = my_builder.read_template_tsv(expressionMatrix_normalized)
df_norm.head()

### 编写完成后应如下图所示
### 列名中，第一列为normalizationMethod, 第二列为cellID, 其余列为基因名。

![title](http://118.190.148.166/biodb/dataset_curation/static/7.png)

### 2.2 从 ../downloaded_data/ 路径中读取原始数据，进行清洗

In [ ]:
df_downloaded = pd.read_csv('../downloaded_data/x.csv')

### 2.3 清洗完成后，确认当前格式与目标格式相同，存储数据

In [ ]:
my_builder.save_template_tsv(df_norm, expressionMatrix_normalized)

### 2.4 存储为sparse matrix格式

normalized一般不需要存储为mtx格式；如有特殊情况咨询管理员后可使用下面的函数

In [ ]:
my_builder.save_template_mtx(mtx=, mtx_name='expressionMatrix_normalized.mtx')

<br><br><br><br><br><br><br><br><br>

# 3. 填写expressionMatrix_TPM.tsv

#### TPM填写注意事项：
1.如果原文只提供了rawCounts矩阵，则使用6中的自动化函数生成

2.如果原文提供了normalized矩阵，则由normalizedMethod推导生成TPM矩阵

### 3.1 读入标准模板查看格式

In [ ]:
df_tpm = my_builder.read_template_tsv(expressionMatrix_TPM)
df_tpm.head()

### 编写完成后应如下图所示
### 列名中，第一列为normalizationMethod(TPM的生成方法), 第二列为cellID, 其余列为基因名。

![title](http://118.190.148.166/biodb/dataset_curation/static/8.png)

### 3.3 将expressionMatrix_normalized.tsv中的数据，或expressionMatrix_rawCounts.tsv 中的数据转换、存入expressionMatrix_TPM.tsv，可参考教程

### 存储TPM文件

In [ ]:
my_builder.save_template_tsv(df_tpm, expressionMatrix_TPM)

### 3.4 存储为sparse matrix格式

由normalized转换而来得到的TPM一般不需要存储为mtx格式；如有特殊情况使用下面的函数

In [ ]:
my_builder.save_template_mtx(mtx=, mtx_name='expressionMatrix_TPM.mtx')

<br><br><br><br><br><br><br><br><br>

# 4. 填写cellAnnotation.tsv

#### 注：如果你的数据找不到cluster信息，请使用我们的邮件模板发邮件给作者询问，如仍无回应，填为notAvailable

### 4.1 读入标准模板查看格式

In [ ]:
df_cell = my_builder.read_template_tsv(cellAnnotation)
df_cell.head()

### 编写完成后应如下图所示

![title](http://118.190.148.166/biodb/dataset_curation/static/5.png)

### 4.2 将expressionMatrix_TPM.tsv 中的cellID填在cellAnnotation.tsv中

In [ ]:
df_tpm = my_builder.read_template_tsv(expressionMatrix_TPM)
df_tpm.head()

In [ ]:
df_cell['cellID'] = df_tpm['cellID'].tolist()

### 4.3 根据对应的cellID填写其他列

#### 注意事项：
#### sampleID需要和cellID一一对应，一般可通过title进行对应，如确实无法对应，填为notAvailable
#### meta字段为选填，填写时注意：
1.根据sample中的信息选择填写，除了模板中给的三个字段示例之外，需要自行添加，一般为其Characteristics的几个属性，可以先浏览GSM的界面查看，确定之后从sample里面摘下来

2.模板中的只是示例，如果没有该字段信息的请删除！不确定要不要填的请咨询管理员，切不可遗漏！

3.字段命名方式参考meta_sourceName，中间不要出现空格和其他连接符号，比如cell type命名为meta_cellType

4.有些数据在文章的supplementary data里面也有关于sample或是细胞的注释信息，可以自行下载并填写进去

In [ ]:
#读取sample注释文件
sample = pd.read_csv('../downloaded_data/GSM_info.tsv'')

In [ ]:
#若无该文件，则使用内置函数获取
sample = my_builder.sample_info(GSE = )

In [ ]:
#填写sample metadata相关信息,字段名前面加上meta_,如meta_sourceName等,以下仅为示例，不是必填！
df_cell['meta_sourceName'] = sample['']
df_cell['meta_tissue'] = sample['']
# .....

In [ ]:
df_cell['clusterID'] = ''
df_cell['clusterName'] = ''
df_cell['sampleID'] = ''
df_cell['cellOntologyName'] = ''
df_cell['cellOntologyID'] = ''
df_cell['FACSMarker'] = ''
df_cell['tSNE1'] = ''# 从文章提供的表格或者其他数据来源中查找tSNE信息填入，若无再使用6中的函数生成
df_cell['tSNE2'] = ''

### 4.4 编辑完后保存

In [ ]:
my_builder.save_template_tsv(df_cell, cellAnnotation)

<br><br><br><br><br><br><br><br><br>

# 5 填写unstructuredData.json

### 5.1 读入并查看模板 

In [ ]:
uns = my_builder.read_template_json(unstructuredData)
uns

### unstructuredData.json 含有两部分，metadata和markerGenes

### 5.2 填写数据

#### metadata填写注意事项：
#### 部分字段会自动填充，可以运行查看，如有错误可以手动改正；其余字段需手动填上
#### libraryPreparationMethod需要填写注释中给定的几个字段之一，如果没有可以咨询管理员确认，不要乱填！也不要硬对上标准字段！

In [ ]:
metadata = uns['metadata']
metadata

In [ ]:
number_of_cells = my_builder.read_template_tsv(cellAnnotation).shape[0]
metadata['numberOfCells'] = number_of_cells

In [ ]:
metadata['subDataset'] = '' # 如果有不只一个part，则根据part_n填为SubDataset-n；只有一个part不填
metadata['description'] = '' #  对subDataset的描述，填写规则看下面注释；只有一个part不填
metadata['title'] = '' # 注意这里填写的是文章的title，不是GSE界面的title！
metadata['authors'] = my_builder.get_metadata('')[2] # 使用内置函数获取，引号内填pubmedID
metadata['accessionNumber'] = ''
metadata['pubmedID'] = 0 # 注意填数字，不是字符串
metadata['keywords'] = my_builder.get_metadata('')[3] # 使用内置函数获取，引号内填pubmedID
metadata['abstract'] = ''
metadata['sourceID'] = ''
metadata['libraryPreparationMethod'] = '' # 从下面注释的对应部分中选填
metadata['sequencingPlatform'] = '' # 'Illumina HiSeq 2500'
metadata['clusteringMethod'] = ''
metadata['biomarkerDerivationMethod'] = '' # 有cluster分类的填: wilcoxon，没有的填: notAvailable
metadata['fastqURL'] = 'https://www.ebi.ac.uk/ena/data/view/PRJNA542142'
metadata['figureURL'] = 'https://media.springernature.com/lw685/springer-static/image/art%3A10.1038%2Fs41590-019-0420-3/MediaObjects/41590_2019_420_Fig1_HTML.png?as=webp' # 使用文章的Graphical Abstract，如果没有则用第一张图Fig.1，链接需能在浏览器中打开
metadata['isFigurePublic'] = ''# 填True or False, 是否对所有网络均公开可见，如一般abstract中的figure为公开的
metadata['taxonomyID'] = 0 # 人填9606， 鼠填10090
metadata['genomeBuild'] = '' # 'hg38'
metadata['annotation'] = '' # 'ensembl_v93'
metadata['journal'] =  my_builder.get_metadata('')[0] # 使用内置函数获取，引号内填pubmedID，如无法获取，从下面注释的对应部分中选填
metadata['publicationDate'] = my_builder.get_metadata('')[1] # 使用内置函数获取，引号内填pubmedID，格式为：2018-03-04
metadata['citation'] = 0 # 引用次数
metadata['tissue'] = [''] # 从下面注释的对应部分中选填,填为list格式
metadata['tissueOntology'] = [''] # 根据tissue的内容填写tissueOntology, 以BTO开头，搜寻网址为：https://www.ebi.ac.uk/ols/ontologies/bto
metadata['clusterAvailability'] = ''# 填True or False，意思是能否找到对应的cluster信息
# 以下字段为文章的研究主题，注意是字符串格式
metadata['disease'] = '' # 填True or False
metadata['methodology'] = ''# 填True or False
metadata['cancer'] = ''# 填True or False
metadata['neuroscience'] = '' # 填True or False
metadata['developmentalBiology'] = ''# 填True or False
metadata['immunology'] = ''# 填True or False
metadata['cellAtlas'] = ''# 填True or False
metadata['tSNEAvailability'] = # 填True or False,意思是能否找到tSNE的坐标信息，同样需要发邮件
metadata['isBadtSNE'] = '' # 填True or False,由6中的tSNEplot画出图形判断，有cluster需要判断，没有的空着

In [ ]:
my_builder.get_metadata('')[3]

libraryPreparationMethod: 10x chromium,drop-seq,microwell-seq,C1 Fluidigm,inDrops,Smart-seq2,Smart-seq,CEL-seq,CEL-seq2,MARS-seq,msSCRB-seq,SCRB-seq.

journal: Cancer cell,Cancer discovery,Cell,Cell stem cell,Cell system,Immmunity,Molecular cell,Nature,Nature biomedical engineering,Nature cell biology,Nature communications,Nature genetics,Nature medicine,Nature methods,Nature neuroscience,Neuron,Science,Science immunology,Science translational medicine,eLife

description: The original article contains ? subdatasets, based on "e.g. different sequencing platforms", "e.g. adult & fatal tissue",. This subdataset is based on "e.g. smartseq-2", "adult tissue", (should match the initial description.)


如果你的数据不符合以上给定字符串，请告知管理员！

In [ ]:
# 查看给定的tissue keywords
my_inspector.tissue_keywords

In [ ]:
# 存储数据
uns['metadata'] = metadata
my_builder.save_template_json(uns, unstructuredData)

### 5.3 填写marker genes，由文章给的数据填写或者邮件发送作者询问

如果给的是geneSymbol,可使用my_builder.calculate_ensemblID(geneSymbol)得到对应的ensemblID；

如果给的是ensemblID,可使用my_builder.calculate_geneSymbol(ensemblID)得到对应的geneSymbol。

注：需要填写taxomomyID并保存，输入输出格式均为list

In [ ]:
# 查看模板格式
uns['markerGenes']

In [ ]:
markerGenes = {}
markerGenes['cluster1'] = {} #cluster1为clusterName，每个clusterName都要有以下五个keywords
# 下面等号右边填为List格式，且长度一致，没有的填notAvailable，也需为list格式
markerGenes['cluster1']['geneSymbol'] = []
markerGenes['cluster1']['ensemblID'] = []
markerGenes['cluster1']['pValue'] = []
markerGenes['cluster1']['statistics'] = [] # 统计量
markerGenes['cluster1']['statisticsType'] = [] # 如wilcoxon

In [ ]:
# 存储数据
uns['markerGenes'] = markerGenes
my_builder.save_template_json(uns, unstructuredData)

<br><br><br><br><br><br><br><br><br>

# 6. 使用脚本自动生成其他项
### 以下几个函数需要按照顺序运行

### 6.1 由raw生成TPM（未提供normalized矩阵的情况下）
 运行该命令需要首先填写unstruc文件中metadata部分的libraryPreparationMethod，如果libraryPreparationMethod在我们给定的标准字段（见上面的注释）中，则直接运行下面的命令，否则需要判断是否为全长测序，并加入参数gene_len = True or False.
 
 如果raw counts存为sparse matrix，也可运行该函数由raw counts得到TPM，同时会自动生成geneAnnotation文件

In [ ]:
# generate TPM from rawCou
my_builder.toTPM() 

### 6.2 生成geneAnnotation文件

#### 如果不能生成首先查下TPM矩阵的列名格式是否正确

In [ ]:
my_builder.generate_geneAnno()

### 6.3 生成tSNE,UMAP

#### 如果出错首先查下TPM格式是否正确，是否存在负值

In [ ]:
# generate dim_red
# 其中tSNE坐标需先从文章中寻找或者发邮件给作者询问，若无可使用下面函数生成
my_builder.calculate_dim_red(tSNE = True, UMAP = True)

### 6.4 Clustering
原文及作者没有提供clusterName时，需运行下面的函数得到聚类结果

In [ ]:
my_builder.calculate_cluster()

#### 画图判断isBadtSNE
如果图上的颜色混杂在一起，则为bad。

In [ ]:
my_builder.tSNEplot()

### 6.5 自动生成其他文件

#### 如果cellAnno文件中的clusterID和clusterName都是notAvailable，则不需运行此函数！

In [ ]:
my_builder.auto_calculation()

<br><br><br><br><br><br><br><br><br>

# 7. 填写README

In [ ]:
my_builder = builder.DatasetBuilder()

#读入模板
Readme = my_builder.read_template_json(readme)
Readme

In [ ]:
#更改信息
Readme['author'] = ''
Readme['date'] = ''
Readme['modificationDate'] = ''
Readme['unfinishedParts'] = [''] #未完成的部分，包括矩阵及cellAnno中的cluster等
Readme['authorComments'] = '' #如果数据分成了几个part，写清楚该part代表什么类型的数据
Readme['otherComments'] = ''

In [ ]:
#保存文件 
my_builder.save_template_json(Readme, readme)

<br><br><br><br><br><br><br><br><br>

# 8. 使用检查器检查

### 8.1 运行如下代码，检查器会提示错误信息

In [ ]:
my_inspector.inspect() #注意看报ERROR的内容！